In [12]:
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_agent
import pandas as pd
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from functools import partial
from pprint import pprint

import getpass

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [13]:


class StudyPlanEvaluation(BaseModel):
    """
    Full evaluation of a student's semester study plan by multiple agents.
    """
    scheduling_score: int = Field(
        description="Score out of 100 given by the Scheduling Agent."
    )
    alignment_score: int = Field(
        description="Score out of 100 given by the Alignment Agent."
    )
    weighted_color: str = Field(
        description="Final approval color code (red/yellow/green) given by main agent."
    )
    scheduling_reasoning: str = Field(
        description="Brief reasoning from Scheduling Agent."
    )
    alignment_reasoning: str = Field(
        description="Brief reasoning from Alignment Agent."
    )
    overall_recommendation: str = Field(
        description="Joint summary or recommendation by main agent (optional)."
    )



In [14]:
@tool
def weighted_score_tool(scheduling_score:int, alignment_score:int) -> str:
    """
    Calculate the weighted average score and return a color code based on the score."""
    w_avg = (0.6*scheduling_score + 0.4*alignment_score)/100
    if 0 <= w_avg <= 45:
        return "red"
    elif 46 <= w_avg <= 75:
        return "yellow"
    else:
        return "green"


In [15]:
# Load your CSV into a pandas DataFrame
course_description_path = 'course_description.csv'
course_masterlist_path = 'course_masterlist.csv'
exams_path = 'exams.csv'
lectures_path = 'lectures.csv'

course_description_df = pd.read_csv(course_description_path)
course_masterlist_df = pd.read_csv(course_masterlist_path)
exams_df = pd.read_csv(exams_path)
lectures_df = pd.read_csv(lectures_path)

def query_dataframe(df: pd.DataFrame,query: str) -> str:
    try:
        result = df.query(query).to_string()
        return result
    except Exception as e:
        return f"Error executing query: {e}"


course_description_query = partial(query_dataframe, course_description_df)
course_masterlist_query = partial(query_dataframe, course_masterlist_df)
exams_query = partial(query_dataframe, exams_df)
lectures_query = partial(query_dataframe, lectures_df)


In [16]:
def load_prompt(path):
    with open(path, 'r') as f:
        return f.read()

scheduling_prompt_text = load_prompt("schedule_prompt.txt")
alignment_prompt_text = load_prompt("alignment_prompt.txt")
print(scheduling_prompt_text)

Given a Study Plan: {study_plan}
To extract semester schedule details for each selected course, use the course_masterlist, exams, and lectures tables.
Evaluate when lectures, exercises, and exams occur for selected courses.
Score out of 100 based on:
- Breaks (<1hr) between lectures/exercises
- Exams too close (<3 days)
- Overlaps/conflicts not tolerated
Explain your score briefly.



In [17]:
@tool
def course_description_tool(query: str) -> str:
    """
    Query the course description dataframe. Accepts a pandas query string."""
    return query_dataframe(df=course_description_df,query=query)

@tool
def course_masterlist_tool(query: str) -> str:
    """
    Query the course masterlist dataframe. Accepts a pandas query string."""
    return query_dataframe(df=course_masterlist_df,query=query)

@tool
def exams_tool(query: str) -> str:
    """
    Query the exams dataframe. Accepts a pandas query string.
    """
    return query_dataframe(df=course_masterlist_df,query=query)
@tool
def lectures_tool(query: str) -> str:
    """
    Query the lectures dataframe. Accepts a pandas query string.
    """
    return query_dataframe(df=lectures_df,query=query)


# Scheduling agent prompt
scheduling_prompt = ChatPromptTemplate.from_messages(('human',scheduling_prompt_text))

# Alignment agent prompt
alignment_prompt = ChatPromptTemplate.from_messages(('human',alignment_prompt_text))

llm = ChatGroq(model="llama-3.3-70b-versatile",
    temperature=0.0,
    max_retries=2)  # Substitute with your preferred model



In [18]:
# Define each agent as a LangChain AgentExecutor, with relevant tools
scheduling_agent = create_agent(model=llm, tools=[exams_tool,lectures_tool])  # Add relevant table tools

@tool("schedule_evaluator", description="Assesses study plan schedule for breaks and conflicts.")
def call_scheduling_agent(study_plan: str) -> dict:
    chain = scheduling_prompt | scheduling_agent
    result = chain.invoke({"study_plan": study_plan})
    # Ensure the output structure matches StudyPlanEvaluation (score and reasoning)
    return {
        "score": result.get("score"),
        "reasoning": result.get("reasoning")
    }

alignment_agent = create_agent(model=llm, tools=[course_description_tool,course_masterlist_tool])  # Add relevant table tools
@tool("alignment_evaluator", description="Evaluates courses alignment with major/minor.")
def call_alignment_agent(study_plan: str) -> dict:
    chain = alignment_prompt | alignment_agent
    result = chain.invoke({"study_plan": study_plan})
    # Ensure the output structure matches StudyPlanEvaluation (score and reasoning)
    return {
        "score": result.get("score"),
        "reasoning": result.get("reasoning")
    }



In [30]:
main_llm = ChatGroq(model="llama-3.3-70b-versatile",
    temperature=0.0,
    max_retries=2)
main_agent_prompt_text = """
You are the study plan supervisor. For the provided study plan, call schedule_evaluator and alignment_evaluator as needed.
Aggregate their results, call weighted_score, and return a structured StudyPlanEvaluation containing:
- scheduling_score, alignment_score, weighted_color
- scheduling_reasoning, alignment_reasoning
- overall_recommendation
"""
# Use from_messages to construct the ChatPromptTemplate (ChatPromptTemplate requires 'messages' when using constructor)
main_agent_prompt = ChatPromptTemplate.from_messages([('system',main_agent_prompt_text),('human',"Given a study plan: \n{study_plan}\n")])
structured_llm_with_tools = main_llm.bind_tools(
    tools=[call_scheduling_agent, call_alignment_agent, weighted_score_tool, StudyPlanEvaluation])
main_agent = create_agent(
    model=structured_llm_with_tools
)
chain = main_agent_prompt | main_agent

In [37]:
def run_evaluation(study_plan: str):
    result = chain.invoke({"study_plan": study_plan})
    # Parse results into StudyPlanEvaluation
    return result

In [45]:
green_case = load_prompt("green_case.txt")
yellow_case = load_prompt("yellow_structured.txt")
red_case = load_prompt("red_structured.txt")
pprint(run_evaluation(yellow_case),indent=2,depth=9)

{ 'messages': [ SystemMessage(content='\nYou are the study plan supervisor. For the provided study plan, call schedule_evaluator and alignment_evaluator as needed.\nAggregate their results, call weighted_score, and return a structured StudyPlanEvaluation containing:\n- scheduling_score, alignment_score, weighted_color\n- scheduling_reasoning, alignment_reasoning\n- overall_recommendation\n', additional_kwargs={}, response_metadata={}, id='b4b0eb6c-2491-4baa-be29-177bacf2043b'),
                HumanMessage(content='Given a study plan: \nStudent Name: Leo\nAcademic Program: Economics Major, International Business Minor\nSemester: Fall 2025\n\nMajor Requirements:\n\nECO-101: Principles of Microeconomics\n\nECO-220: Intermediate Macroeconomics\n\nMinor Requirements:\n\nIB-400: International Business Strategy\n\nElective Courses:\n\nMTH-215: Linear Algebra (Mathematics Department)\n\nMKT-205: Principles of Marketing (Marketing Department)\n\nCHM-101: General Chemistry I (Chemistry Departme